In [1]:
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
import spacy

In [2]:
female_name_dict = []
with open('data/female_name_dict.txt', 'r', encoding='utf-8') as file:
    for line in file:
        female_name_dict.append(line.strip())
female_name_dict=set(female_name_dict)
male_name_dict = []
with open('data/male_name_dict.txt', 'r', encoding='utf-8') as file:
    for line in file:
        male_name_dict.append(line.strip())
male_name_dict=set(male_name_dict)

In [ ]:
# data preprocessing, very time-consuming
# Set the stopwords
stop_words = set(stopwords.words('english'))

# Define a list of words representing females/males
female_words = set(['woman', 'girl', 'lady', 'female', 'daughter', 'mother', 'sister','girlfriend','maid','virgin','damsel','madame','senorita','princess','queen','her'])
female_names_words = female_words | female_name_dict
male_words = set(['man', 'boy', 'gentleman', 'male', 'son', 'father', 'brother','boyfriend','beau','husband','dude','lad','prince','king','him'])
male_names_words = male_words | male_name_dict
remove_dict=(stop_words | female_names_words | male_names_words)
#from https://www.merriam-webster.com/thesaurus/

def preprocess_text(text):
    # lower the words and remove punctuation
    text_token = word_tokenize(text)
    tagged = pos_tag(text_token)
    adjectives = [word for word, pos in tagged if (pos.startswith('JJ'))]
    text = [word.lower() for word in adjectives if word.isalpha()]
    # tokenize, replace words representing females with 'she' and males with 'he', and remove stopwords.
    return [word for word in text if word not in remove_dict ]

# 初始化一个空字符串用于存储结果
long_string = ""
documents=[]

# load summaries
with open('data/plot_summaries.txt', 'r', encoding='utf-8') as file:
    for line in file:
        text = line
        documents.append(preprocess_text(text))
documents = [word for sublist in documents for word in sublist]

In [ ]:
# remove repetitive words
print(len(documents))
documents=list(set(documents))
print(len(documents))

In [20]:
# to save time if you want to make change on the adj lists
''' documents=[]
with open('data/adjectives.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # 使用 strip() 方法去除每行末尾的换行符并添加到列表
        documents.append(line.strip())'''

In [25]:
# remove some words in case of incorrect classification
lines = []

# file from https://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/
with open('data/female name.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line=line.lower()
        lines.append(line.strip())
documents = set(documents)-(set(lines) & set(documents))

lines = []
with open('data/male name.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line=line.lower()
        lines.append(line.strip())
documents = set(documents)-(set(lines) & set(documents))

lines = []
with open('data/names from movies.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line=line.lower()
        lines.append(line.strip())
documents = set(documents)-(set(lines) & set(documents))
print(len(documents))
other_common_words = set(['child', 'orphan', 'baby', 'girls', 'lover', 'mute','child', 'orphan', 'bride', 
                          'baby', 'girls', 'lover', 'mute', 'housekeeper', 'cousin', 'neighbour', 'parent', 
                          'sibling', 'pregnant', 'housewife', 'relative', 'servant', 'waif', 'grandson', 
                          'stranger', 'courtesan', 'servant','cousin', 'neighbour', 'parent', 'sibling', 
                          'housewife','nurse', 'stepsister', 'housekeeper','grandson','servant','grandchildren',
                          'relatives', 'uncles', 'orphans', 'childbirth', 'scoundrel', 'foreigner', 'stepfamily', 
                          'tuberculosis', 'mallaya', 'triplet', 'grandchildren', 'outcast', 'neighbours', 'huanhuan', 
                          'lakshmiammal', 'jaipal', 'himal', 'pasarian', 'leukemia'])
documents = set(documents)-(set(documents) & set(other_common_words))

13335
13322


In [26]:
# save data
with open('data/adjectives.txt', 'w', encoding='utf-8') as file:
    for item in documents:
        file.write(item + '\n')